# Исследование надёжности заёмщиков

Для кредитного отдела банка нужно разобраться, какие факторы влияют на погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В таблице 12 столбцов, в них указаны данные по заемщикам, которые могут влиять на их платежеспособность. 

- children — количество детей в семье
- days_employed — трудовой стаж в днях
- dob_days — возраст клиента в годах
- education — образование клиента
- education_id — идентификатор образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — доход в месяц
- purpose — цель получения кредита

Строки в датасете, это заявки на выдачу кредита, пронумерованные по порядку. Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но перед началом анализа, нужно провести предобрабоку этих данных, так как встречаются пропуски и странные значения, например, отрицательные значения в столбце days_employed. Качество необработанных данных данных может негативно сказаться на результатах исследования.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df['days_employed'] = df['days_employed'].apply(abs) #убираем отрицательные значения из столбца
df.loc[df['days_employed'] >30000, 'days_employed'] = df.loc[df['days_employed'] >30000, 'days_employed'] / 24 #убираем большие значения в столбце
df.sort_values('days_employed').tail(10) #проверка
    

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


Для начала уберем отрицательные значения из стажа, просто взяв этот столбец по модулю. Так же в этом столбце неправдоподобно большие данные. Это могло было произойти из-за того, что часть данных для датасета предоставлялось не в днях, а часах. Исходя из здравого смысла, определим их, как данные выше порогового значения 30 000 дней. Все значения выше разделим на 24 и получим все данные в часах. Изменения в данных стажа не влияют на результаты исследования, поэтому такой подход допустим.

In [3]:
print('Пропуски до:')
print(df.isna().sum()) #количество пропусков в столбцах
print()
print(df.loc[df['days_employed'].isna(), ['dob_years', 'education', 'income_type']].head(10)) #попытка понять причирну пропусков
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) #замена пропусков медианным значением
print()
print('Медианный доход по типам занятости, руб/мес')
print() # Замена пропусков медианой по группам
for income in df['income_type'].unique():
    median = df.loc[df['income_type'] == income, 'total_income'].median()
    print(f'{income}, - {median}')
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income), 'total_income'] = median
print()    
print('Пропуски после:')
df[['days_employed', 'total_income']].isna().sum() #Проверка

Пропуски до:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

    dob_years education  income_type
12         65   среднее    пенсионер
26         41   среднее  госслужащий
29         63   среднее    пенсионер
41         50   среднее  госслужащий
55         54   среднее    пенсионер
65         21   среднее    компаньон
67         52    высшее    пенсионер
72         32    высшее  госслужащий
82         50    высшее    сотрудник
83         52   среднее    сотрудник

Медианный доход по типам занятости, руб/мес

сотрудник, - 142594.39684740017
пенсионер, - 118514.48641164352
компаньон, - 172357.95096577113
госслужащий, - 150447.9352830068
безработный, - 131339.7516762103
предприниматель, - 499163.1449470857
студент, - 98201.6253

days_employed    0
total_income     0
dtype: int64

**Вывод**

Пропуски содержатся всего в двух стодбцах: стаж (days_employed) и доход (total_income). Судя по их равному количеству, это одни и те же заемщики. Мы посмотрели первые 10 строк с пропущенными данными и не обнаружили очевидных закономерностей, почему эти заемщики предпочли не указывать стаж и размер дохода.  Пропуски в этих столбцах могли появиться как специально (люди не захотели делиться этими данными), были пропущены случайно либо из-за технических сложностей. Пропуски составляют примерно 10% от всей совокупности данных, это довольно значимое число. Было решено заменить пропуски в стаже их медианным значением для всех значений данного столбца. А пропуске в доходе медианным доходом для каждого типа занятости, так как этот столбец влияет на результаты исследования и для него нужен более точный подход.

### Замена типа данных

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head() #проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Числовые данные в столбцах стаж в днях и доход были переведены в целочисленный для того, чтобы их было комфортней воспринимать. Применялся метод as_type()

### Обработка дубликатов

In [5]:
print('Уникальные значения в столбце Образование, до:')
print(df['education'].unique()) # 
df['education'] = df['education'].str.lower()
print()
print('Уникальные значения в столбце Образование, после:')
df['education'].unique() 


Уникальные значения в столбце Образование, до:
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные значения в столбце Образование, после:


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Варианты данных в столбце образование привели к единообразным значениям, заменив прописные буквы на строчные

In [6]:

print(f'Количество строк-дубликатов до: {df.duplicated(keep = False).sum()}')
df[df.duplicated(keep = False)].sort_values(by = list(df.columns)) # keep = False выведет все совпадения

Количество строк-дубликатов до: 137


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,2194,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
18328,0,2194,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
4216,0,2194,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9013,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования
14432,2,2194,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования
11033,2,2194,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу


In [7]:
df = df.drop_duplicates().reset_index(drop = True) #удаление дубликатов
print(f'Количество строк-дубликатов после: {df.duplicated(keep = False).sum()}')

Количество строк-дубликатов после: 0


**Вывод**

Было выявлено 137 полных строк-дубликатов, они могли появиться как из-за человеческого фактора - какие-то строки были занесены дважды, так и технических сбоев. Были удалены методом drop.duplucates() c обновлением индексов.

### Лемматизация

In [8]:
from pymystem3 import Mystem #импорт библиотеки лемм на русском языке
m = Mystem() 
#' '.join(purpose_uniq)
text = ' '.join(df['purpose'].unique()) #преобразование списка значений стобца purpose в строку

In [9]:
lemmas = m.lemmatize(text) #получение списка лемм
print('Леммы, использованные в столбце purpose:')
print(lemmas)

Леммы, использованные в столбце purpose:
['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование',

In [10]:
from collections import Counter #подсчет упоминания каждой из лемм
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [11]:
def purpose_cat(text): # группипровка целей по категориям
    lemmas = m.lemmatize(text)
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'жилье' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'авто'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'прочее'
 
df['purpose_cat'] = df['purpose'].apply(purpose_cat)
print('Разбивка целей на категории с частотой использования')
df['purpose_cat'].value_counts() 

Разбивка целей на категории с частотой использования


недвижимость    10811
авто             4306
образование      4013
прочее           2324
Name: purpose_cat, dtype: int64

**Вывод**

Чтобы проанализировать, есть ли связь между целями, на которые берется кредит, и его возвратом в срок необходимо разбить цели (столбец purpose) на группы. Этот трудоемкий процесс возможно автоматизировать с помощью лемматизации. Выделяются основы слов, используемых в purpose и в зависимости от частоты повторения лемм, выделяются отдельные категории целей. В результате все цели были сгурппированы в четыре категории, которые можно будет использовать в дальнейшем

### Категоризация данных

In [12]:
print('Уникальные значения children до:')
print(df['children'].unique())
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 2)
print('Уникальные значения children после:')
print(df['children'].unique()) #проверка

Уникальные значения children до:
[ 1  0  3  2 -1  4 20  5]
Уникальные значения children после:
[1 0 3 2 4 5]


Для того, чтобы категоризировать количество детей, сначало нужно посмотреть варианты заполнения данного столбца. Допустим, что -1 это означает отсутствие детей, 20 - это опечатка, и имелось в виду 2. И проведем соответствующие изменения

Заявки по уровню дохода были разбиты на 5 равных групп с помощью метода qcut

In [13]:
df['total_income_gr'] = pd.qcut(df['total_income'], 5)
df['total_income_gr'].value_counts()

(214618.2, 2265604.0]    4291
(161335.0, 214618.2]     4291
(98537.6, 132134.4]      4291
(20666.999, 98537.6]     4291
(132134.4, 161335.0]     4290
Name: total_income_gr, dtype: int64

**Вывод**

Категоризация данных - это необходимы шаг перед оперделением зависимостей и взаимосвязей между разфакторами и возвратностью кредитов. Цели были уже сгруппированны в разделе Лемматизация. В этом разделе данные были сгруппированы по количеству детей и уровню дохода.

<div class="alert alert-danger">
<font size="5"><b>Комментарий ревьюера</b></font>

Ошибка: В данных есть словари. Попробуй их обнаружить и выделить. Это довольно распространенный способ хранения информации. Мне важно понять, что он тобой освоен. Заметь, что в основной таблице мы имеем по 2 параметра с образованием и семейным статусом. Именно по ним и стоит выделить словари. 

</div>

In [14]:
print(df['family_status_id'].unique())
print(df['family_status'].unique())
df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')


[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


Здесь мы проверили категоральные переменные и удостоверились, что столбцы family_status и family_status_id заполнены правильно

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
print('зависимость между количеством детей и возвратом кредита в срок')
df_pivot = df.pivot_table(index = 'children', columns = 'debt', values = 'days_employed', aggfunc = 'count')
df_pivot.columns = ['no_debt', 'debt'] #переименуем столбцы для наглядности
df_pivot['debt_%'] = df_pivot['debt']/(df_pivot['debt']+df_pivot['no_debt'])*100
df_pivot

зависимость между количеством детей и возвратом кредита в срок


,no_debt,debt,debt_%
children,,,
0,13074.0,1064.0,7.525817
1,4364.0,444.0,9.234609
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


Чтобы ответить на вопрос "Есть ли зависимость между количеством детей и возвратом кредита в срок?", создадим сводную таблицу, где все заявки на выдачу кредита, будут разбиты на группы по количеству детей, по строкам, а в столбцах будет разбивка этих же заявок, на были ли задержки по возврату кредита (debt: значение 1 в столбце debt исходного датафрейма) или нет (no_debt: значение 0 в столбце debt исходного датафрейма).
В столбце debt_% посчитаем % проблемных задолженностей в общем количестве заявок. Это будет наш ключевой показатель, анализ которого в разрезе отдельных групп будет ответом на вопросы. 
Сводные таблица для анализа других факторов будут сделаны аналогично.

In [16]:
df_sum = df_pivot['debt'].sum()+df_pivot['no_debt'].sum() #проверка правильно ли сделан пивот - подсчет количества строк
if df.shape[0] == df_sum:
    print('сводная таблица сделана правильно')

сводная таблица сделана правильно


**Вывод**

Наличие детей в целом негативно влияет на возврат кредита в срок. Проблемных кредитов среди бездетных заявок 7,5%. Тогда как наличие одного ребенка повышает это процент до 9,2, а двух до 9,4. Появление третьего ребенка в семье немного улучшает картину и понижает его до 8,2.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
print('зависимость между семейным положением и возвратом кредита в срок')
df_pivot = df.pivot_table(index = 'family_status', columns = 'debt', values = 'days_employed', aggfunc = 'count')
df_pivot.columns = ['no_debt', 'debt'] #переименуем столбцы для наглядности
df_pivot['debt_%'] = df_pivot['debt']/(df_pivot['debt']+df_pivot['no_debt'])*100
df_pivot

зависимость между семейным положением и возвратом кредита в срок


,no_debt,debt,debt_%
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**

Самый высокий процент невозвратов у свободных людей либо не оформивших свои отношения оффициально. Причем у людей уже побывавших в браке (сейчас в разводе или вдовцы/вдовы) этот процент наиболее низкий. Достоверность этих выводов довольно сомнительна, так как такие категориях, как вдовцы и в разводе малочисленны и небольшая разница в процентах может быть не из-за реальных причин, а качества данных и погрешностей вычислений. Такие результаты могут косвенно говорить о влиянии возраста на возвратность кредитов. Т.е. молодые люди (не женатые, живущие в гражданском браке) имеют больше проблем с возвратом кредитов, чем более взрослые.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
print('зависимость между уровнем дохода и возвратом кредита в срок')
df_pivot = df.pivot_table(index = 'total_income_gr', columns = 'debt', values = 'days_employed', aggfunc = 'count')
df_pivot.columns = ['no_debt', 'debt'] #переименуем столбцы для наглядности
df_pivot['debt_%'] = df_pivot['debt']/(df_pivot['debt']+df_pivot['no_debt'])*100
df_pivot

зависимость между уровнем дохода и возвратом кредита в срок


,no_debt,debt,debt_%
total_income_gr,,,
"(20666.999, 98537.6]",3947,344,8.016779
"(98537.6, 132134.4]",3930,361,8.412957
"(132134.4, 161335.0]",3915,375,8.741259
"(161335.0, 214618.2]",3930,361,8.412957
"(214618.2, 2265604.0]",3991,300,6.991377


**Вывод**

Самый низкий процент невозвратов у лиц с доходом от 215 т.р/мес и выше). Чем больше доход, тем меньше процент невозвратов.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
print('зависимость между целями и возвратом кредита в срок')
df_pivot = df.pivot_table(index = 'purpose_cat', columns = 'debt', values = 'days_employed', aggfunc = 'count')
df_pivot.columns = ['no_debt', 'debt'] #переименуем столбцы для наглядности
df_pivot['debt_%'] = df_pivot['debt']/(df_pivot['debt']+df_pivot['no_debt'])*100
df_pivot

зависимость между целями и возвратом кредита в срок


,no_debt,debt,debt_%
purpose_cat,,,
авто,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
прочее,2138,186,8.003442


**Вывод**

Самый низкий процент невозвратов наблюдается у кредитов взятых для покупки недвижимости. Самый высокий  - у взятых для покупки автомобиля и получения образования. Данный результат также может косвенно говорить о влиянии возраста на возвратность кредитов. Предположительно, что кредиты на образование берут молодые люди. 

## Шаг 4. Общий вывод

Мы провели анализ влияют ли такие факторы как наличие детей, семейной положение, уровень дохода и цели кредитования на возвратность кредитов. В ходе исследования мы выявили, что у различных категорий заемщиков процент невозвратных кредитов колеблется не сильно от 6% до 9% от общего числа кредитов. Т.е ярко выраженных зависимостей не наблюдается. Портрет идеального заемщика: вдовец без детей с высоким доходом берет кредит на покупку недвижимости. Нужно принимать во внимание, что небольшой разброс в пределах 1-2% может быть вызван погрешностями измерений и неточности предоставленных данных, а не влиянием определенных факторов. Так же по результатам проделанной работы можно судить о необходимости проведения дополнительного исследования о влиянии возраста заемщиков на возвратность кредитов.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.